In [10]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import platform
import matplotlib.pyplot as plt
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium import webdriver

In [11]:
driver = webdriver.Chrome('C:/Program Files/Google/Chrome/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

In [12]:
driver.find_element_by_id("electionType1").click()

In [13]:
driver.find_element_by_id("electionName").send_keys("제19대")

In [14]:
driver.find_element_by_id("electionCode").send_keys("대통령선거")

In [15]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [16]:
def get_num(a):
    return float(re.split('\(', a)[0].replace(',', ''))

In [17]:
wait = WebDriverWait(driver, 10)

def move_sido(name):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH, make_xpath)))
    driver.find_element_by_xpath(make_xpath).click()

In [18]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append((each[2]))
        data['moon'].append(get_num(each[3]))
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

In [19]:
election_result_raw = {'광역시도' : [],
                       '시군' : [],
                       'pop' : [],
                       'moon' : [],
                       'hong' : [],
                       'ahn' : [] }

In [20]:
for each_sido in sido_names_values:
    move_sido(each_sido)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')

    df = pd.read_html(str(table))

    append_data(df, each_sido, election_result_raw)

In [21]:
election_result = pd.DataFrame(election_result_raw,
                               columns=['광역시도', '시군', 'pop', 'moon','hong','ahn'])
election_result

,광역시도,시군,pop,moon,hong,ahn
0,서울특별시,종로구,102566,42512.0,22325.0,22313.0
1,서울특별시,중구,82852,34062.0,17901.0,19372.0
2,서울특별시,용산구,148157,58081.0,35230.0,32109.0
3,서울특별시,성동구,203175,86686.0,40566.0,45674.0
4,서울특별시,광진구,240030,105512.0,46368.0,52824.0
...,...,...,...,...,...,...
420,제주특별자치도,관악구,351240,159854.0,59914.0,79495.0
421,제주특별자치도,서초구,292916,106416.0,74891.0,63977.0
422,제주특별자치도,강남구,365642,128927.0,97639.0,80201.0
423,제주특별자치도,송파구,441198,177328.0,98549.0,99018.0


In [22]:
election_result.to_csv('./data_saved/05. election_result.csv', encoding='utf-8', sep=',')

In [23]:
driver.close()

In [24]:
election_result = pd.read_csv('./data_saved/05. election_result.csv', encoding='utf-8',
                              index_col=0)
election_result.head()

,광역시도,시군,pop,moon,hong,ahn
0,서울특별시,종로구,102566,42512.0,22325.0,22313.0
1,서울특별시,중구,82852,34062.0,17901.0,19372.0
2,서울특별시,용산구,148157,58081.0,35230.0,32109.0
3,서울특별시,성동구,203175,86686.0,40566.0,45674.0
4,서울특별시,광진구,240030,105512.0,46368.0,52824.0


In [25]:
sido_candi =  election_result['광역시도']
sido_candi = [name[:2] if name[:2]
                                          in ['서울','부산','대구','광주','인천','대전','울산']
                                    else '' for name in sido_candi]

In [26]:
def cut_char_sigu(name):
    return name if len(name)==2 else name[:-1]

In [27]:
sigun_candi = ['']*len(election_result)

for n in election_result.index:
    each = election_result['시군'][n]
    if each[:2] in ['수원', '성남','안양','안산','고양',
                            '용인','청주','천안','전주','포항','창원']:
        sigun_candi[n] = re.split('시', each)[0]+' '+ \
                                                        cut_char_sigu(re.split('시', each)[1])
    else:
        sigun_candi[n] = cut_char_sigu(each)

sigun_candi


['종로',
 '중구',
 '용산',
 '성동',
 '광진',
 '동대문',
 '중랑',
 '성북',
 '강북',
 '도봉',
 '노원',
 '은평',
 '서대문',
 '마포',
 '양천',
 '강서',
 '구로',
 '금천',
 '영등포',
 '동작',
 '관악',
 '서초',
 '강남',
 '송파',
 '강동',
 '종로',
 '중구',
 '용산',
 '성동',
 '광진',
 '동대문',
 '중랑',
 '성북',
 '강북',
 '도봉',
 '노원',
 '은평',
 '서대문',
 '마포',
 '양천',
 '강서',
 '구로',
 '금천',
 '영등포',
 '동작',
 '관악',
 '서초',
 '강남',
 '송파',
 '강동',
 '종로',
 '중구',
 '용산',
 '성동',
 '광진',
 '동대문',
 '중랑',
 '성북',
 '강북',
 '도봉',
 '노원',
 '은평',
 '서대문',
 '마포',
 '양천',
 '강서',
 '구로',
 '금천',
 '영등포',
 '동작',
 '관악',
 '서초',
 '강남',
 '송파',
 '강동',
 '종로',
 '중구',
 '용산',
 '성동',
 '광진',
 '동대문',
 '중랑',
 '성북',
 '강북',
 '도봉',
 '노원',
 '은평',
 '서대문',
 '마포',
 '양천',
 '강서',
 '구로',
 '금천',
 '영등포',
 '동작',
 '관악',
 '서초',
 '강남',
 '송파',
 '강동',
 '종로',
 '중구',
 '용산',
 '성동',
 '광진',
 '동대문',
 '중랑',
 '성북',
 '강북',
 '도봉',
 '노원',
 '은평',
 '서대문',
 '마포',
 '양천',
 '강서',
 '구로',
 '금천',
 '영등포',
 '동작',
 '관악',
 '서초',
 '강남',
 '송파',
 '강동',
 '종로',
 '중구',
 '용산',
 '성동',
 '광진',
 '동대문',
 '중랑',
 '성북',
 '강북',
 '도봉',
 '노원',
 '은평',
 '서대문',
 '마포',
 '양천',
 '강

In [28]:
ID_candi = [sido_candi[n]+' '+sigun_candi[n] for n in range(0,len(sigun_candi))]

ID_candi = [name[1:] if name[0]==' ' else name for name in ID_candi]
ID_candi = [name[:2] if name[:2]=='세종' else name for name in ID_candi]

ID_candi

['서울 종로',
 '서울 중구',
 '서울 용산',
 '서울 성동',
 '서울 광진',
 '서울 동대문',
 '서울 중랑',
 '서울 성북',
 '서울 강북',
 '서울 도봉',
 '서울 노원',
 '서울 은평',
 '서울 서대문',
 '서울 마포',
 '서울 양천',
 '서울 강서',
 '서울 구로',
 '서울 금천',
 '서울 영등포',
 '서울 동작',
 '서울 관악',
 '서울 서초',
 '서울 강남',
 '서울 송파',
 '서울 강동',
 '부산 종로',
 '부산 중구',
 '부산 용산',
 '부산 성동',
 '부산 광진',
 '부산 동대문',
 '부산 중랑',
 '부산 성북',
 '부산 강북',
 '부산 도봉',
 '부산 노원',
 '부산 은평',
 '부산 서대문',
 '부산 마포',
 '부산 양천',
 '부산 강서',
 '부산 구로',
 '부산 금천',
 '부산 영등포',
 '부산 동작',
 '부산 관악',
 '부산 서초',
 '부산 강남',
 '부산 송파',
 '부산 강동',
 '대구 종로',
 '대구 중구',
 '대구 용산',
 '대구 성동',
 '대구 광진',
 '대구 동대문',
 '대구 중랑',
 '대구 성북',
 '대구 강북',
 '대구 도봉',
 '대구 노원',
 '대구 은평',
 '대구 서대문',
 '대구 마포',
 '대구 양천',
 '대구 강서',
 '대구 구로',
 '대구 금천',
 '대구 영등포',
 '대구 동작',
 '대구 관악',
 '대구 서초',
 '대구 강남',
 '대구 송파',
 '대구 강동',
 '인천 종로',
 '인천 중구',
 '인천 용산',
 '인천 성동',
 '인천 광진',
 '인천 동대문',
 '인천 중랑',
 '인천 성북',
 '인천 강북',
 '인천 도봉',
 '인천 노원',
 '인천 은평',
 '인천 서대문',
 '인천 마포',
 '인천 양천',
 '인천 강서',
 '인천 구로',
 '인천 금천',
 '인천 영등포',
 '인천 동작',
 '인천 관악',
 '인천 서초',
 '인천 강남',
 '인천 송파'

In [29]:
election_result['ID'] = ID_candi
election_result.head(10)

,광역시도,시군,pop,moon,hong,ahn,ID
0,서울특별시,종로구,102566,42512.0,22325.0,22313.0,서울 종로
1,서울특별시,중구,82852,34062.0,17901.0,19372.0,서울 중구
2,서울특별시,용산구,148157,58081.0,35230.0,32109.0,서울 용산
3,서울특별시,성동구,203175,86686.0,40566.0,45674.0,서울 성동
4,서울특별시,광진구,240030,105512.0,46368.0,52824.0,서울 광진
5,서울특별시,동대문구,236092,98958.0,51631.0,53359.0,서울 동대문
6,서울특별시,중랑구,265706,111450.0,56545.0,62778.0,서울 중랑
7,서울특별시,성북구,295866,129263.0,57584.0,66518.0,서울 성북
8,서울특별시,강북구,210614,89645.0,42268.0,51669.0,서울 강북
9,서울특별시,도봉구,229233,94898.0,47461.0,55600.0,서울 도봉


In [30]:
election_result[['rate_moon','rate_hong','rate_ahn']] = \
            election_result[['moon','hong','ahn']].div(election_result['pop'],  axis=0)
election_result[['rate_moon','rate_hong','rate_ahn']] *= 100
election_result.head()

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn
0,서울특별시,종로구,102566,42512.0,22325.0,22313.0,서울 종로,41.448433,21.766472,21.754773
1,서울특별시,중구,82852,34062.0,17901.0,19372.0,서울 중구,41.111862,21.605996,23.381451
2,서울특별시,용산구,148157,58081.0,35230.0,32109.0,서울 용산,39.202333,23.778829,21.672280
3,서울특별시,성동구,203175,86686.0,40566.0,45674.0,서울 성동,42.665682,19.966039,22.480128
4,서울특별시,광진구,240030,105512.0,46368.0,52824.0,서울 광진,43.957839,19.317585,22.007249


In [31]:
election_result.sort_values(['rate_moon'], ascending=[False]).head(10)
election_result.sort_values(['rate_hong'], ascending=[False]).head(10)
election_result.sort_values(['rate_ahn'], ascending=[False]).head(10)

draw_korea = pd.read_csv('./data_saved/05. draw_korea.csv', encoding='utf-8', index_col=0)

draw_korea.head()

,y,x,ID
0,0,7,철원
1,0,8,화천
2,0,9,양구
3,0,10,고성(강원)
4,1,3,양주


In [32]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

{'가평',
 '강릉',
 '강진',
 '거제',
 '거창',
 '경산',
 '경주',
 '계룡',
 '고령',
 '고성(강원)',
 '고성(경남)',
 '고양 덕양',
 '고양 일산동',
 '고양 일산서',
 '고창',
 '고흥',
 '곡성',
 '공주',
 '과천',
 '광명',
 '광양',
 '광주',
 '광주 광산',
 '광주 남구',
 '광주 동구',
 '광주 북구',
 '광주 서구',
 '괴산',
 '구례',
 '구리',
 '구미',
 '군산',
 '군위',
 '군포',
 '금산',
 '김제',
 '김천',
 '김포',
 '김해',
 '나주',
 '남양주',
 '남원',
 '남해',
 '논산',
 '단양',
 '담양',
 '당진',
 '대구 남구',
 '대구 달서',
 '대구 달성',
 '대구 동구',
 '대구 북구',
 '대구 서구',
 '대구 수성',
 '대전 대덕',
 '대전 동구',
 '대전 서구',
 '대전 유성',
 '동두천',
 '동해',
 '목포',
 '무안',
 '무주',
 '문경',
 '밀양',
 '보령',
 '보성',
 '보은',
 '봉화',
 '부산 금정',
 '부산 기장',
 '부산 남구',
 '부산 동구',
 '부산 동래',
 '부산 부산진',
 '부산 북구',
 '부산 사상',
 '부산 사하',
 '부산 서구',
 '부산 수영',
 '부산 연제',
 '부산 영도',
 '부산 해운대',
 '부안',
 '부여',
 '부천 소사',
 '부천 오정',
 '부천 원미',
 '사천',
 '산청',
 '삼척',
 '상주',
 '서귀포',
 '서산',
 '서천',
 '성남 분당',
 '성남 수정',
 '성남 중원',
 '성주',
 '세종',
 '속초',
 '수원 권선',
 '수원 영통',
 '수원 장안',
 '수원 팔달',
 '순창',
 '순천',
 '시흥',
 '신안',
 '아산',
 '안동',
 '안산 단원',
 '안산 상록',
 '안성',
 '안양 동안',
 '안양 만안',
 '양구',
 '양산',
 '양양',
 '양주',
 '양평

In [33]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

{'강남',
 '강동',
 '강북',
 '강서',
 '관악',
 '광주 강남',
 '광주 강동',
 '광주 강북',
 '광주 강서',
 '광주 관악',
 '광주 광진',
 '광주 구로',
 '광주 금천',
 '광주 노원',
 '광주 도봉',
 '광주 동대문',
 '광주 동작',
 '광주 마포',
 '광주 서대문',
 '광주 서초',
 '광주 성동',
 '광주 성북',
 '광주 송파',
 '광주 양천',
 '광주 영등포',
 '광주 용산',
 '광주 은평',
 '광주 종로',
 '광주 중구',
 '광주 중랑',
 '광진',
 '구로',
 '금천',
 '노원',
 '대구 강남',
 '대구 강동',
 '대구 강북',
 '대구 강서',
 '대구 관악',
 '대구 광진',
 '대구 구로',
 '대구 금천',
 '대구 노원',
 '대구 도봉',
 '대구 동대문',
 '대구 동작',
 '대구 마포',
 '대구 서대문',
 '대구 서초',
 '대구 성동',
 '대구 성북',
 '대구 송파',
 '대구 양천',
 '대구 영등포',
 '대구 용산',
 '대구 은평',
 '대구 종로',
 '대구 중랑',
 '대전 강남',
 '대전 강동',
 '대전 강북',
 '대전 강서',
 '대전 관악',
 '대전 광진',
 '대전 구로',
 '대전 금천',
 '대전 노원',
 '대전 도봉',
 '대전 동대문',
 '대전 동작',
 '대전 마포',
 '대전 서대문',
 '대전 서초',
 '대전 성동',
 '대전 성북',
 '대전 송파',
 '대전 양천',
 '대전 영등포',
 '대전 용산',
 '대전 은평',
 '대전 종로',
 '대전 중랑',
 '도봉',
 '동대문',
 '동작',
 '마포',
 '부산 강남',
 '부산 강동',
 '부산 강북',
 '부산 관악',
 '부산 광진',
 '부산 구로',
 '부산 금천',
 '부산 노원',
 '부산 도봉',
 '부산 동대문',
 '부산 동작',
 '부산 마포',
 '부산 서대문',
 '부산 서초',
 '부산 성동',
 '부산 성북',
 '부산 송파

In [34]:
election_result[election_result['ID'] == '고성']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [35]:
election_result.loc[125, 'ID'] = '고성(강원)'
election_result.loc[233, 'ID'] = '고성(경남)'

election_result[election_result['시군'] == '고성군']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [36]:
election_result[election_result['광역시도'] == '경상남도']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn
375,경상남도,종로구,102566,42512.0,22325.0,22313.0,종로,41.448433,21.766472,21.754773
376,경상남도,중구,82852,34062.0,17901.0,19372.0,중구,41.111862,21.605996,23.381451
377,경상남도,용산구,148157,58081.0,35230.0,32109.0,용산,39.202333,23.778829,21.672280
378,경상남도,성동구,203175,86686.0,40566.0,45674.0,성동,42.665682,19.966039,22.480128
379,경상남도,광진구,240030,105512.0,46368.0,52824.0,광진,43.957839,19.317585,22.007249
380,경상남도,동대문구,236092,98958.0,51631.0,53359.0,동대문,41.915016,21.869017,22.600935
381,경상남도,중랑구,265706,111450.0,56545.0,62778.0,중랑,41.944856,21.281040,23.626866
382,경상남도,성북구,295866,129263.0,57584.0,66518.0,성북,43.689711,19.462865,22.482475
383,경상남도,강북구,210614,89645.0,42268.0,51669.0,강북,42.563647,20.068941,24.532557
384,경상남도,도봉구,229233,94898.0,47461.0,55600.0,도봉,41.398054,20.704262,24.254798


In [37]:
election_result.loc[228, 'ID'] = '창원 합포'
election_result.loc[229, 'ID'] = '창원 회원'
election_result[election_result['광역시도'] == '경상남도']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn
375,경상남도,종로구,102566,42512.0,22325.0,22313.0,종로,41.448433,21.766472,21.754773
376,경상남도,중구,82852,34062.0,17901.0,19372.0,중구,41.111862,21.605996,23.381451
377,경상남도,용산구,148157,58081.0,35230.0,32109.0,용산,39.202333,23.778829,21.672280
378,경상남도,성동구,203175,86686.0,40566.0,45674.0,성동,42.665682,19.966039,22.480128
379,경상남도,광진구,240030,105512.0,46368.0,52824.0,광진,43.957839,19.317585,22.007249
380,경상남도,동대문구,236092,98958.0,51631.0,53359.0,동대문,41.915016,21.869017,22.600935
381,경상남도,중랑구,265706,111450.0,56545.0,62778.0,중랑,41.944856,21.281040,23.626866
382,경상남도,성북구,295866,129263.0,57584.0,66518.0,성북,43.689711,19.462865,22.482475
383,경상남도,강북구,210614,89645.0,42268.0,51669.0,강북,42.563647,20.068941,24.532557
384,경상남도,도봉구,229233,94898.0,47461.0,55600.0,도봉,41.398054,20.704262,24.254798


In [38]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

{'가평',
 '강릉',
 '강진',
 '거제',
 '거창',
 '경산',
 '경주',
 '계룡',
 '고령',
 '고양 덕양',
 '고양 일산동',
 '고양 일산서',
 '고창',
 '고흥',
 '곡성',
 '공주',
 '과천',
 '광명',
 '광양',
 '광주',
 '광주 광산',
 '광주 남구',
 '광주 동구',
 '광주 북구',
 '광주 서구',
 '괴산',
 '구례',
 '구리',
 '구미',
 '군산',
 '군위',
 '군포',
 '금산',
 '김제',
 '김천',
 '김포',
 '김해',
 '나주',
 '남양주',
 '남원',
 '남해',
 '논산',
 '단양',
 '담양',
 '당진',
 '대구 남구',
 '대구 달서',
 '대구 달성',
 '대구 동구',
 '대구 북구',
 '대구 서구',
 '대구 수성',
 '대전 대덕',
 '대전 동구',
 '대전 서구',
 '대전 유성',
 '동두천',
 '동해',
 '목포',
 '무안',
 '무주',
 '문경',
 '밀양',
 '보령',
 '보성',
 '보은',
 '봉화',
 '부산 금정',
 '부산 기장',
 '부산 남구',
 '부산 동구',
 '부산 동래',
 '부산 부산진',
 '부산 북구',
 '부산 사상',
 '부산 사하',
 '부산 서구',
 '부산 수영',
 '부산 연제',
 '부산 영도',
 '부산 해운대',
 '부안',
 '부여',
 '부천 소사',
 '부천 오정',
 '부천 원미',
 '사천',
 '산청',
 '삼척',
 '상주',
 '서귀포',
 '서산',
 '서천',
 '성남 분당',
 '성남 수정',
 '성남 중원',
 '성주',
 '세종',
 '속초',
 '수원 권선',
 '수원 영통',
 '수원 장안',
 '수원 팔달',
 '순창',
 '순천',
 '시흥',
 '신안',
 '아산',
 '안동',
 '안산 단원',
 '안산 상록',
 '안성',
 '안양 동안',
 '안양 만안',
 '양구',
 '양산',
 '양양',
 '양주',
 '양평',
 '여수',
 '여주',
 '연천'

In [39]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

{'강남',
 '강동',
 '강북',
 '강서',
 '관악',
 '광주 강남',
 '광주 강동',
 '광주 강북',
 '광주 강서',
 '광주 관악',
 '광주 광진',
 '광주 구로',
 '광주 금천',
 '광주 노원',
 '광주 도봉',
 '광주 동대문',
 '광주 동작',
 '광주 마포',
 '광주 서대문',
 '광주 서초',
 '광주 성동',
 '광주 성북',
 '광주 송파',
 '광주 양천',
 '광주 영등포',
 '광주 용산',
 '광주 은평',
 '광주 종로',
 '광주 중구',
 '광주 중랑',
 '광진',
 '구로',
 '금천',
 '노원',
 '대구 강남',
 '대구 강동',
 '대구 강북',
 '대구 강서',
 '대구 관악',
 '대구 광진',
 '대구 구로',
 '대구 금천',
 '대구 노원',
 '대구 도봉',
 '대구 동대문',
 '대구 동작',
 '대구 마포',
 '대구 서대문',
 '대구 서초',
 '대구 성동',
 '대구 성북',
 '대구 송파',
 '대구 양천',
 '대구 영등포',
 '대구 용산',
 '대구 은평',
 '대구 종로',
 '대구 중랑',
 '대전 강남',
 '대전 강동',
 '대전 강북',
 '대전 강서',
 '대전 관악',
 '대전 광진',
 '대전 구로',
 '대전 금천',
 '대전 노원',
 '대전 도봉',
 '대전 동대문',
 '대전 동작',
 '대전 마포',
 '대전 서대문',
 '대전 서초',
 '대전 성동',
 '대전 성북',
 '대전 송파',
 '대전 양천',
 '대전 영등포',
 '대전 용산',
 '대전 은평',
 '대전 중랑',
 '도봉',
 '동대문',
 '동작',
 '마포',
 '부산 강남',
 '부산 강동',
 '부산 강북',
 '부산 관악',
 '부산 광진',
 '부산 구로',
 '부산 금천',
 '부산 노원',
 '부산 도봉',
 '부산 동대문',
 '부산 동작',
 '부산 마포',
 '부산 서대문',
 '부산 서초',
 '부산 성동',
 '부산 성북',
 '부산 송파',
 '부산 양천

In [40]:
election_result[election_result['시군'] == '부천시']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [41]:
election_result.tail()

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn
420,제주특별자치도,관악구,351240,159854.0,59914.0,79495.0,관악,45.511331,17.057852,22.632673
421,제주특별자치도,서초구,292916,106416.0,74891.0,63977.0,서초,36.329869,25.567398,21.841415
422,제주특별자치도,강남구,365642,128927.0,97639.0,80201.0,강남,35.260446,26.703442,21.934296
423,제주특별자치도,송파구,441198,177328.0,98549.0,99018.0,송파,40.192385,22.336683,22.442985
424,제주특별자치도,강동구,297476,122511.0,65893.0,67499.0,강동,41.183490,22.150695,22.690570


In [42]:
ahn_tmp = election_result.loc[85, 'ahn']/3
hong_tmp = election_result.loc[85, 'hong']/3
moon_tmp = election_result.loc[85, 'moon']/3
pop_tmp = election_result.loc[85, 'pop']/3

rate_moon_tmp = election_result.loc[85, 'rate_moon']
rate_hong_tmp = election_result.loc[85, 'rate_hong']
rate_ahn_tmp = election_result.loc[85, 'rate_ahn']

election_result.loc[250] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp,
                           '경기도', '부천시', '부천 소사',
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[251] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp,
                           '경기도', '부천시', '부천 오정',
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[252] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp,
                           '경기도', '부천시', '부천 원미',
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]

In [43]:
election_result[election_result['시군'] == '부천시']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [44]:
election_result.drop([85], inplace=True)
election_result[election_result['시군'] == '부천시']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [45]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

{'가평',
 '강릉',
 '강진',
 '거제',
 '거창',
 '경산',
 '경주',
 '계룡',
 '고령',
 '고양 덕양',
 '고양 일산동',
 '고양 일산서',
 '고창',
 '고흥',
 '곡성',
 '공주',
 '과천',
 '광명',
 '광양',
 '광주',
 '광주 광산',
 '광주 남구',
 '광주 동구',
 '광주 북구',
 '광주 서구',
 '괴산',
 '구례',
 '구리',
 '구미',
 '군산',
 '군위',
 '군포',
 '금산',
 '김제',
 '김천',
 '김포',
 '김해',
 '나주',
 '남양주',
 '남원',
 '남해',
 '논산',
 '단양',
 '담양',
 '당진',
 '대구 남구',
 '대구 달서',
 '대구 달성',
 '대구 동구',
 '대구 북구',
 '대구 서구',
 '대구 수성',
 '대전 대덕',
 '대전 동구',
 '대전 서구',
 '대전 유성',
 '동두천',
 '동해',
 '목포',
 '무안',
 '무주',
 '문경',
 '밀양',
 '보령',
 '보성',
 '보은',
 '봉화',
 '부산 금정',
 '부산 기장',
 '부산 남구',
 '부산 동구',
 '부산 동래',
 '부산 부산진',
 '부산 북구',
 '부산 사상',
 '부산 사하',
 '부산 서구',
 '부산 수영',
 '부산 연제',
 '부산 영도',
 '부산 해운대',
 '부안',
 '부여',
 '사천',
 '산청',
 '삼척',
 '상주',
 '서귀포',
 '서산',
 '서천',
 '성남 분당',
 '성남 수정',
 '성남 중원',
 '성주',
 '세종',
 '속초',
 '수원 권선',
 '수원 영통',
 '수원 장안',
 '수원 팔달',
 '순창',
 '순천',
 '시흥',
 '신안',
 '아산',
 '안동',
 '안산 단원',
 '안산 상록',
 '안성',
 '안양 동안',
 '안양 만안',
 '양구',
 '양산',
 '양양',
 '양주',
 '양평',
 '여수',
 '여주',
 '연천',
 '영광',
 '영덕',
 '영동',
 '영암',


In [46]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

{'강남',
 '강동',
 '강북',
 '강서',
 '관악',
 '광주 강남',
 '광주 강동',
 '광주 강북',
 '광주 강서',
 '광주 관악',
 '광주 광진',
 '광주 구로',
 '광주 금천',
 '광주 노원',
 '광주 도봉',
 '광주 동대문',
 '광주 동작',
 '광주 마포',
 '광주 서대문',
 '광주 서초',
 '광주 성동',
 '광주 성북',
 '광주 송파',
 '광주 양천',
 '광주 영등포',
 '광주 용산',
 '광주 은평',
 '광주 종로',
 '광주 중구',
 '광주 중랑',
 '광진',
 '구로',
 '금천',
 '노원',
 '대구 강남',
 '대구 강동',
 '대구 강북',
 '대구 강서',
 '대구 관악',
 '대구 광진',
 '대구 구로',
 '대구 금천',
 '대구 노원',
 '대구 도봉',
 '대구 동대문',
 '대구 동작',
 '대구 마포',
 '대구 서대문',
 '대구 서초',
 '대구 성동',
 '대구 성북',
 '대구 송파',
 '대구 양천',
 '대구 영등포',
 '대구 용산',
 '대구 은평',
 '대구 종로',
 '대구 중랑',
 '대전 강남',
 '대전 강동',
 '대전 강북',
 '대전 강서',
 '대전 관악',
 '대전 광진',
 '대전 구로',
 '대전 금천',
 '대전 노원',
 '대전 도봉',
 '대전 동대문',
 '대전 동작',
 '대전 마포',
 '대전 서대문',
 '대전 서초',
 '대전 성동',
 '대전 성북',
 '대전 송파',
 '대전 양천',
 '대전 영등포',
 '대전 용산',
 '대전 은평',
 '대전 중랑',
 '도봉',
 '동대문',
 '동작',
 '마포',
 '부산 강남',
 '부산 강동',
 '부산 강북',
 '부산 관악',
 '부산 광진',
 '부산 구로',
 '부산 금천',
 '부산 노원',
 '부산 도봉',
 '부산 동대문',
 '부산 동작',
 '부산 마포',
 '부산 서대문',
 '부산 서초',
 '부산 성동',
 '부산 성북',
 '부산 송파',
 '부산 양천

In [47]:
final_elect_data = pd.merge(election_result, draw_korea, how='left', on=['ID'])
final_elect_data.head()

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn,y,x
0,서울특별시,종로구,102566.0,42512.0,22325.0,22313.0,서울 종로,41.448433,21.766472,21.754773,4.0,6.0
1,서울특별시,중구,82852.0,34062.0,17901.0,19372.0,서울 중구,41.111862,21.605996,23.381451,5.0,6.0
2,서울특별시,용산구,148157.0,58081.0,35230.0,32109.0,서울 용산,39.202333,23.778829,21.672280,6.0,6.0
3,서울특별시,성동구,203175.0,86686.0,40566.0,45674.0,서울 성동,42.665682,19.966039,22.480128,5.0,7.0
4,서울특별시,광진구,240030.0,105512.0,46368.0,52824.0,서울 광진,43.957839,19.317585,22.007249,6.0,7.0


In [48]:
final_elect_data['moon_vs_hong'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_hong']
final_elect_data['moon_vs_ahn'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_ahn']
final_elect_data['ahn_vs_hong'] = final_elect_data['rate_ahn'] - \
                                                                final_elect_data['rate_hong']
final_elect_data.head()

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn,y,x,moon_vs_hong,moon_vs_ahn,ahn_vs_hong
0,서울특별시,종로구,102566.0,42512.0,22325.0,22313.0,서울 종로,41.448433,21.766472,21.754773,4.0,6.0,19.681961,19.693661,-0.011700
1,서울특별시,중구,82852.0,34062.0,17901.0,19372.0,서울 중구,41.111862,21.605996,23.381451,5.0,6.0,19.505866,17.730411,1.775455
2,서울특별시,용산구,148157.0,58081.0,35230.0,32109.0,서울 용산,39.202333,23.778829,21.672280,6.0,6.0,15.423503,17.530053,-2.106549
3,서울특별시,성동구,203175.0,86686.0,40566.0,45674.0,서울 성동,42.665682,19.966039,22.480128,5.0,7.0,22.699643,20.185554,2.514089
4,서울특별시,광진구,240030.0,105512.0,46368.0,52824.0,서울 광진,43.957839,19.317585,22.007249,6.0,7.0,24.640253,21.950590,2.689664


In [49]:
final_elect_data.sort_values(['moon_vs_hong'], ascending=[False]).head(10)

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn,y,x,moon_vs_hong,moon_vs_ahn,ahn_vs_hong
70,대구광역시,관악구,351240.0,159854.0,59914.0,79495.0,대구 관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
20,서울특별시,관악구,351240.0,159854.0,59914.0,79495.0,서울 관악,45.511331,17.057852,22.632673,7.0,5.0,28.453479,22.878658,5.574821
244,강원도,관악구,351240.0,159854.0,59914.0,79495.0,관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
369,경상북도,관악구,351240.0,159854.0,59914.0,79495.0,관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
144,대전광역시,관악구,351240.0,159854.0,59914.0,79495.0,대전 관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
194,세종특별자치시,관악구,351240.0,159854.0,59914.0,79495.0,관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
45,부산광역시,관악구,351240.0,159854.0,59914.0,79495.0,부산 관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
119,광주광역시,관악구,351240.0,159854.0,59914.0,79495.0,광주 관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
219,경기도,관악구,351240.0,159854.0,59914.0,79495.0,관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821
269,충청북도,관악구,351240.0,159854.0,59914.0,79495.0,관악,45.511331,17.057852,22.632673,NaN,NaN,28.453479,22.878658,5.574821


In [50]:
final_elect_data.sort_values(['moon_vs_hong'], ascending=[True]).head(10)

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn,y,x,moon_vs_hong,moon_vs_ahn,ahn_vs_hong
72,대구광역시,강남구,365642.0,128927.0,97639.0,80201.0,대구 강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
22,서울특별시,강남구,365642.0,128927.0,97639.0,80201.0,서울 강남,35.260446,26.703442,21.934296,8.0,6.0,8.557004,13.32615,-4.769146
171,울산광역시,강남구,365642.0,128927.0,97639.0,80201.0,울산 강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
321,전라북도,강남구,365642.0,128927.0,97639.0,80201.0,강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
371,경상북도,강남구,365642.0,128927.0,97639.0,80201.0,강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
296,충청남도,강남구,365642.0,128927.0,97639.0,80201.0,강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
96,인천광역시,강남구,365642.0,128927.0,97639.0,80201.0,인천 강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
47,부산광역시,강남구,365642.0,128927.0,97639.0,80201.0,부산 강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
396,경상남도,강남구,365642.0,128927.0,97639.0,80201.0,강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146
146,대전광역시,강남구,365642.0,128927.0,97639.0,80201.0,대전 강남,35.260446,26.703442,21.934296,NaN,NaN,8.557004,13.32615,-4.769146


In [51]:
BORDER_LINES = [
    [(5, 1), (5,2), (7,2), (7,3), (11,3), (11,0)], # 인천
    [(5,4), (5,5), (2,5), (2,7), (4,7), (4,9), (7,9),
     (7,7), (9,7), (9,5), (10,5), (10,4), (5,4)], # 서울
    [(1,7), (1,8), (3,8), (3,10), (10,10), (10,7),
     (12,7), (12,6), (11,6), (11,5), (12, 5), (12,4),
     (11,4), (11,3)], # 경기도
    [(8,10), (8,11), (6,11), (6,12)], # 강원도
    [(12,5), (13,5), (13,4), (14,4), (14,5), (15,5),
     (15,4), (16,4), (16,2)], # 충청북도
    [(16,4), (17,4), (17,5), (16,5), (16,6), (19,6),
     (19,5), (20,5), (20,4), (21,4), (21,3), (19,3), (19,1)], # 전라북도
    [(13,5), (13,6), (16,6)], # 대전시
    [(13,5), (14,5)], #세종시
    [(21,2), (21,3), (22,3), (22,4), (24,4), (24,2), (21,2)], #광주
    [(20,5), (21,5), (21,6), (23,6)], #전라남도
    [(10,8), (12,8), (12,9), (14,9), (14,8), (16,8), (16,6)], #충청북도
    [(14,9), (14,11), (14,12), (13,12), (13,13)], #경상북도
    [(15,8), (17,8), (17,10), (16,10), (16,11), (14,11)], #대구
    [(17,9), (18,9), (18,8), (19,8), (19,9), (20,9), (20,10), (21,10)], #부산
    [(16,11), (16,13)], #울산
    [(27,5), (27,6), (25,6)],]

In [52]:
final_elect_data.to_csv('./data_saved/final_elect_data', encoding='utf-8', sep=',')
